<a href="https://colab.research.google.com/github/saklain31/Fantasy-Premier-Leauge-Prediction/blob/master/FPL_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import numpy as np

url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()
json.keys()

In [ ]:
elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])
teams_df

In [ ]:
slim_elements_df = elements_df[['id','second_name','team','element_type','selected_by_percent','now_cost','minutes','transfers_in','value_season','total_points']]
print(slim_elements_df.head())
slim_elements_df['position'] = slim_elements_df.element_type.map(elements_types_df.set_index('id').singular_name)
slim_elements_df['team'] = slim_elements_df.team.map(teams_df.set_index('id').name)
slim_elements_df['value'] = slim_elements_df.value_season.astype(float)
slim_elements_df.head()
slim_elements_df.sort_values('value',ascending=False)


slim_elements_df = slim_elements_df.loc[slim_elements_df.value > 0]
# slim_elements_df
print(type(slim_elements_df))

In [ ]:
def top_points_players():
  df = slim_elements_df.sort_values('total_points',ascending=False)
  print(df.columns)
  return df.set_index('id').T.to_dict('dict')

def top_roi_players():
  df = slim_elements_df.sort_values('value',ascending=False)
  print(df.columns)
  return df.T.to_dict('dict')

In [ ]:
roi_players = top_roi_players()
top_players = top_points_players()

# for i in roi_players:
#   print(roi_players[i]['second_name'], roi_players[i]['team'])

# for i in top_players:
#   print(top_players[i]['second_name'], top_players[i]['team'])

# print(len(roi_players))
# roi_players[384]
print(roi_players)
count = 0 

roi_players_dict = {}
roi_player_id_key = {}
 
for key, value in roi_players.items():
  # print(key, value)
  roi_players_dict[count] = value
  roi_player_id_key[value['id']] = value
  count +=1

# print(count)
# roi_players_dict

In [ ]:
roi_players = top_roi_players()
top_players = top_points_players()

roi_player_count = len(roi_players)

final_team = {}
dp_dict={}

def copy_values(dict2):
  dict1={}
  for key in dict2:
    dict1[key]=dict2[key]
  return dict1

def hash_func(players_from_team):
  val=0
  for team in players_from_team:
    val*=4
    val+=players_from_team[team]
  return val

def knapsack(budget, positions, n, is_main_team, main_team_id, player_teams):
  if n<0:
    flag=False
    for key in positions:
      if(positions[key]!=0):
        flag=True
    if flag:
      return -10000
    return 0
  
  if budget<0:
    return -10000

  hash = hash_func(player_teams)
  key=(budget, positions['Goalkeeper'], positions['Midfielder'], positions['Defender'], positions['Forward'], n, hash)
  

  if key in dp_dict:
    return dp_dict[key]

  current_player = roi_players_dict[n]
  position = current_player['position']
  team=current_player['team']
  with_player = 0
  copy_positions =copy_values(positions)

  with_players_team = copy_values(player_teams)
  without_player_team = copy_values(player_teams)

  if positions[position] > 0 and roi_players_dict[n]['id'] not in main_team_id and player_teams[team]<=3:
    temp_positions=copy_values(positions)
    temp_positions[position] = temp_positions[position] - 1 
    
    with_players_team[team] +=1

    with_player = knapsack(budget-float(current_player['now_cost']), temp_positions, n-1, is_main_team, main_team_id, with_players_team)+current_player['value']  
    without_player = knapsack(budget, copy_positions, n-1, is_main_team, main_team_id, without_player_team)
    if float(with_player) > float(without_player):
      dp_dict[key]=float(with_player)
    else:
      dp_dict[key]=float(without_player)
  else:
    dp_dict[key]=knapsack(budget, copy_positions, n-1, is_main_team, main_team_id, without_player_team)
  return dp_dict[key]


def print_team(budget, positions, n, optimal_team, clubs, id_list, is_main_team, main_team_id, player_teams):
  if n<0:
    # print(optimal_team)
    print(id_list)
    return optimal_team, clubs, id_list

  copyw=copy_values(positions)
  copywo=copy_values(positions)

  current_player = roi_players_dict[n]
  position = current_player['position']
  team = current_player['team']

  with_players_team = copy_values(player_teams)
  without_player_team = copy_values(player_teams)

  with_players_team[team] +=1

  copyw[roi_players_dict[n]['position']]-=1

  if((knapsack(budget-float(roi_players_dict[n]['now_cost']), copyw, n-1, is_main_team, main_team_id, with_players_team)+float(roi_players_dict[n]['value']))>knapsack(budget, copywo, n-1, is_main_team, main_team_id, without_player_team)) and roi_players_dict[n]['id'] not in main_team_id and player_teams[team]<=3:
    optimal_team.append(roi_players_dict[n]['second_name'])
    clubs.append(roi_players_dict[n]['team'])
    id_list.append(roi_players_dict[n]['id'])
    return print_team(budget-float(roi_players_dict[n]['now_cost']), copyw, n-1, optimal_team, clubs, id_list, is_main_team, main_team_id, with_players_team)
  else:
    return print_team(budget, copywo, n-1, optimal_team, clubs, id_list, is_main_team, main_team_id, without_player_team)



# positions = {'Goalkeeper': 1, 'Midfielder': 2, 'Defender': 1, 'Forward': 0}
# main_id = [244, 37, 89, 272, 303, 460, 366, 81, 355, 353, 250, 457, 259, 70, 96]
# knap = knapsack(170, positions, roi_player_count-1, False, main_id)
# print(knap)

# optimal_team, clubs, id_list = print_team(170, positions, roi_player_count-1, [], [], [], False, main_id)
# print("optimal_team",optimal_team)
# print("clubs",clubs)
# print("id_list", id_list)

In [ ]:
team_list = []
for key,value in roi_players_dict.items():
  team_list.append(value['team'])
team_list = list(set(team_list))

players_from_team = {}
for i in team_list:
  players_from_team[i] = 0

main_positions = {'Goalkeeper': 1, 'Midfielder': 3, 'Defender': 4, 'Forward': 3}
# optimal_main_team, main_team_clubs, main_id_list, optimal_subs, subs_clubs, sub_id_list = create_team(600, main_positions, roi_player_count - 1)
ks = knapsack(600, main_positions, roi_player_count - 1, True, [], players_from_team)
print(ks)
print(main_positions)
print(players_from_team)
optimal_main_team, main_team_clubs, main_id_list = print_team(600, main_positions, roi_player_count - 1, [], [], [], True, [], players_from_team)


print(main_id_list)
print(optimal_main_team)
print(main_team_clubs)



In [ ]:
GK_LIMIT = 2
MID_LIMIT = 5
DEF_LIMIT = 5
FWD_LIMIT = 3
BUDGET_LIMIT = 1000

team_list = []
for key,value in roi_players_dict.items():
  team_list.append(value['team'])
team_list = list(set(team_list))

players_from_team = {}
for i in team_list:
  players_from_team[i] = 0

def create_team(main_team_budget, main_team_positions, player_count):
  print(players_from_team)
  main_team_roi = knapsack(main_team_budget, main_team_positions, player_count, True, [], players_from_team)
  print(main_team_roi)
  optimal_main_team, main_team_clubs, main_id_list = print_team(main_team_budget, main_team_positions, player_count, [], [], [], True, [])

  global dp_dict   
  temp_dp_dict = copy_values(dp_dict)
  dp_dict={}

  sub_positions = {'Goalkeeper': GK_LIMIT - main_team_positions['Goalkeeper'] , 'Midfielder': MID_LIMIT - main_team_positions['Midfielder'], 'Defender': DEF_LIMIT - main_team_positions['Defender'] , 'Forward': FWD_LIMIT - main_team_positions['Forward']}
  sub_budget = BUDGET_LIMIT - main_team_budget

  print(sub_budget, sub_positions, main_id_list)
  sub_roi = knapsack(sub_budget, sub_positions, player_count, False, main_id_list)
  print(sub_roi)

  optimal_subs, subs_clubs, sub_id_list = print_team(sub_budget, sub_positions, player_count, [], [], [], False, main_id_list)
  dp_dict = copy_values(temp_dp_dict)
  return optimal_main_team, main_team_clubs, main_id_list, optimal_subs, subs_clubs, sub_id_list


main_positions = {'Goalkeeper': 1, 'Midfielder': 3, 'Defender': 4, 'Forward': 3}
optimal_main_team, main_team_clubs, main_id_list, optimal_subs, subs_clubs, sub_id_list = create_team(600, main_positions, roi_player_count - 1)
print(main_id_list)
print(optimal_main_team)
print(main_team_clubs)
print(sub_id_list)
print(optimal_subs)
print(subs_clubs)




In [ ]:
team_list = []
for key,value in roi_players_dict.items():
  team_list.append(value['team'])
team_list = list(set(team_list))

players_from_team = {}
for i in team_list:
  players_from_team[i] = 0

players_from_team